# Code for Question 3

In [1]:
from ipywidgets import interact, RadioButtons, IntSlider, Output, Layout
import json
import pandas
from tabulate import tabulate
import matplotlib.pyplot as plt
import seaborn as sns

## Import Data

In [2]:
get_season_data(2017, "../ift6758/data/")

NameError: name 'get_season_data' is not defined

In [3]:
year = 2017
filepath = "../ift6758/data"
DIRECTORY  = f"{filepath}/JSON/{year}/"
IMG_PATH = "../figures/nhl_rink.png"

## Game IDs
The first 4 digits identify the season of the game (ie. 2017 for the 2017-2018 season). The next 2 digits give the type of game, where 01 = preseason, 02 = regular season, 03 = playoffs, 04 = all-star. The final 4 digits identify the specific game number. For regular season and preseason games, this ranges from 0001 to the number of games played. (1271 for seasons with 31 teams (2017 and onwards) and 1230 for seasons with 30 teams). For playoff games, the 2nd digit of the specific number gives the round of the playoffs, the 3rd digit specifies the matchup, and the 4th digit specifies the game (out of 7).

## Implementation: Interactive Debugging Tool

In [5]:
#CHOOSE GAME TYPE
type_selector = RadioButtons(
    #options=['preseason', 'regular season', 'playoffs', 'all-star'],
    options=['regular season', 'playoffs'],
    value='regular season', # Defaults to 'pineapple'
    layout={'width': 'max-content'}, # If the items' names are long
    description='Game Type:',
    disabled=False,
    continuous_update=True
)

#CHOOSE GAME ID
game_selector = IntSlider(
    value=1,
    min=1,
    max=1271,
    step=1,
    description='Game #:',
    continuous_update=False, #only reassign value when the slider stops moving
    layout=Layout(width='50%')
)

#display both widgets
display(type_selector, game_selector)

#init game type as 02 which is regular season
type_game = "02"

#LINK game selector to the game type selector
def update_game_selector_range(change):
    global type_game
    if change['new'] == 'regular season':
        max_game = 1271
        type_game = "02"
    elif change['new'] == 'playoffs':
        max_game = 20 #TODO
        type_game = "03"
    else:
        raise Exception("Sorry, game type not recognized")  
    #change range of the game selector
    game_selector.max = max_game

type_selector.observe(update_game_selector_range, 'value')

#Load the JSON when the game range is chosen
output = Output()
display(output)

event_data = {}
data = {}

def on_game_id_change(change):
    with output:
        output.clear_output()
        output2.clear_output()
        
        game_number = str(change['new']).zfill(4)
        game_id = f"{year}{type_game}{game_number}"
        path = f"{DIRECTORY}{game_id}.json"
        #print(path)
        print('                                         ')
        global data
        global event_data
        with open(path, encoding='utf-8') as f:
            data = json.load(f)
        #Print game details
        print("Game ID: ",data['gamePk'])
        print(data['gameData']['teams']['home']['name']," (HOME)   VS.   ",data['gameData']['teams']['away']['name']," (AWAY)")
        print("Start Date: ",data['gameData']['datetime']['dateTime'])
        print("Location: ",data['gameData']['venue']['name'])
        
        data_home_by_teamSkaterStats = data['liveData']['boxscore']['teams']['home']['teamStats']['teamSkaterStats']
        data_away_by_teamSkaterStats = data['liveData']['boxscore']['teams']['away']['teamStats']['teamSkaterStats']
        table = [["", "TEAM", "GOALS", "SHOTS", "PIM", "FACEOFF WIN (%)", "BLOCKS", "HITS"],
                ['HOME', 
                 data['liveData']['boxscore']['teams']['home']['team']['abbreviation'],
                 data_home_by_teamSkaterStats['goals'],
                 data_home_by_teamSkaterStats['shots'],
                 data_home_by_teamSkaterStats['pim'],
                 data_home_by_teamSkaterStats['faceOffWinPercentage'],
                 data_home_by_teamSkaterStats['blocked'],
                 data_home_by_teamSkaterStats['hits']
                ],
                ['AWAY',
                 data['liveData']['boxscore']['teams']['away']['team']['abbreviation'],
                 data_away_by_teamSkaterStats['goals'],
                 data_away_by_teamSkaterStats['shots'],
                 data_away_by_teamSkaterStats['pim'],
                 data_away_by_teamSkaterStats['faceOffWinPercentage'],
                 data_away_by_teamSkaterStats['blocked'],
                 data_away_by_teamSkaterStats['hits']
                ]
               ]
        print('                                         ')
        print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))
        
        #save the event data for later
        event_data = data['liveData']['plays']['allPlays']

game_selector.observe(on_game_id_change, names='value')

###SECOND PART: Field visualisation

event_selector = IntSlider(
    value=1,
    min=1,
    max=2000,
    step=1,
    description='Event #:',
    continuous_update=False, #only reassign value when the slider stops moving
    layout=Layout(width='50%')
)

#display both widgets
display(event_selector)

#LINK game selector to the game type selector
def update_event_selector_range(change):
    global event_data
    event_selector.max = len(event_data)

game_selector.observe(update_event_selector_range, 'value')


##part3
#Load the JSON when the game range is chosen
output2 = Output()
display(output2)

def on_event_id_change(change):
    with output2:
        
        output2.clear_output()
        
        event_id = change['new']
        global event_data
        
        #print("event_id: ",event_id - 1)
        #print("event_id: ",len(event_data))
        
        event = event_data[event_id - 1]
        #print(event)
        _period_num = event['about']['ordinalNum']
        _period_time = event['about']['periodTime']
        _goals_away =  event['about']['goals']['away']
        _goals_home =  event['about']['goals']['home']
        _desc = event['result']['description']
        
        if "x" in event['coordinates']:
            _x, _y = event['coordinates']['x'], event['coordinates']['y']
        else: 
            _x, _y = 0, 0
        
        #create figure
        img = plt.imread(IMG_PATH)
        
        fig, ax = plt.subplots(figsize=(10, 5))
        sns.scatterplot(x=[_x], y=[_y], ax=ax, color = 'k', marker = 'X', s=350)
        plt.xlim(-100, 100)
        plt.ylim(-42.5, 42.5)
        plt.xlabel("feet")
        plt.ylabel("feet")
        plt.title(_desc + '\n' + 
                  "HOME : " + str(_goals_home) + "  ---  AWAY : " + str(_goals_away) + '\n' + 
                  _period_num + ' Period: ' + _period_time) # You can comment this line out if you don't need title
        ax.imshow(img, extent=[-100, 100, -42.5, 42.5], aspect='auto')
        plt.show()

event_selector.observe(on_event_id_change, names='value')

RadioButtons(description='Game Type:', layout=Layout(width='max-content'), options=('regular season', 'playoff…

IntSlider(value=1, continuous_update=False, description='Game #:', layout=Layout(width='50%'), max=1271, min=1…

Output()

IntSlider(value=1, continuous_update=False, description='Event #:', layout=Layout(width='50%'), max=2000, min=…

Output()

.